# Toronto Bike Share Analysis



In [34]:
import os
from dotenv import load_dotenv
import psycopg
from psycopg import sql, OperationalError, ProgrammingError, Error

# Load Environment Variables
load_dotenv()
if os.environ['ENV_FLG']: print('Environment variables loaded properly.')
else: print('ERROR: Environment variables failed to load.');

Environment variables loaded properly.


In [35]:
def run_query(query):
    """
    Queries the local PostgreSQL database containing bike share data.
    Args:
        query (string): SQL query to be executed
    Returns:
        result ([]): array of rows from the database
    """
    try:
        conn = psycopg.connect(
            dbname = os.environ['DB_NAME'],
            user = os.environ['DB_USER'],
            password = os.environ['DB_PASS'],
            host = os.environ['DB_HOST'],
            port = os.environ['DB_PORT']
        )
        
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        print("Connection successful")

    except ProgrammingError as e:
        print(f"ProgrammingError: Query error. Details: {e}")
    except Error as e:
        print(f"Database Error: An unexpected error occurred. Details: {e}")
    finally:
        # Ensure resources are cleaned up
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
        if 'conn' in locals() and conn is not None:
            conn.close()
        return rows

In [36]:
test = run_query("SELECT * FROM bike_trips LIMIT 5;")

Connection successful
